In [18]:

import json
import pandas as pd
from urllib.parse import urlparse

def extract_parts(url):
    parsed = urlparse(url)
    # Extract subdomain (e.g., 'pop', 'globalnation', 'business')
    subdomain = parsed.netloc.split('.')[0]
    origin = parsed.netloc.split('.')[1] if len(parsed.netloc.split('.')) > 1 else ''

    # Split the path: ['', 'article_id', 'article-slug']
    path_parts = parsed.path.strip('/').split('/', 1)

    article_id = path_parts[0] if path_parts else ''
    slug = path_parts[1] if len(path_parts) > 1 else ''

    return {
        'subdomain': subdomain,
        'origin': origin,
        'article_id': article_id,
        'slug': slug
    }

# Step 1: Load the JSON file
with open('news_articles.json', 'r') as f:
    data = json.load(f)

# Step 2: Flatten the structure
records = []
for category, dates in data.items():
    for date, urls in dates.items():
        for url in urls:
            records.append({
                'category': category,
                'date': date,
                'url': url
            })

# Step 3: Convert to DataFrame
df = pd.DataFrame(records)

# Optional: Convert date column to datetime type
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

df['id'] = df['url'].apply(lambda x: extract_parts(x)['subdomain'] + ':' + extract_parts(x)['article_id'] + ':' + extract_parts(x)['slug'])


In [19]:
from util.biquery import create_or_update_table

create_or_update_table(
    df,
    'ph_news',
    'articles'
)

INFO - Table `articles` in dataset `ph_news` updated with mode `append`.
